In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.5 MB/s eta 0:00:00


In [ ]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3639M  100 3639M    0     0  36.3M      0  0:01:40  0:01:40 --:--:-- 37.2M


In [ ]:
!unzip nature_12K.zip



In [ ]:
import wandb

wandb.login(key="8d6c17aa48af2229c26cbc16513ef266358c0b96")
wandb.init(project="Assignment-02")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m076 (saisreeram). Use `wandb login --relogin` to force relogin


In [ ]:
baseDir = "inaturalist_12K"

trainDir = baseDir+"/train/"

testDir = baseDir+"/val/"

outputclasses=["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]


In [ ]:
!pip install pytorch_lightning
import pytorch_lightning as pl


In [ ]:
import torch 
import pytorch_lightning as pl
import torch.nn as nn
from torch.nn  import functional
from pytorch_lightning.loggers import WandbLogger
# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, WandbCallback


class CNN(pl.LightningModule):
  def __init__(self,filters,activation,BatchNorm,dropout,learning_rate):
    super(CNN,self).__init__()
    self.learning_rate = learning_rate
    layers = []
    layers.append(nn.Conv2d(3,filters[0],kernel_size = 3,stride = 1,padding = 0))
    layers.append(nn.MaxPool2d(kernel_size = 2,stride = 1))
    layers.append(activation)

    for i in range(0,3):
      layers.append(nn.Conv2d(filters[i],filters[i+1],kernel_size = 3,stride = 1,padding = 0))
      layers.append(nn.MaxPool2d(kernel_size = 2,stride = 1))
      layers.append( activation)

    layers.append(nn.Conv2d(filters[3],filters[4],kernel_size = 3,stride = 1,padding = 0))
    layers.append(nn.MaxPool2d(kernel_size = 2,stride = 1))
    layers.append( activation)
    layers.append(nn.Flatten())
    if(BatchNorm == "Yes"):
      layers.append(nn.BatchNorm1d(1858592))
    layers.append(nn.Dropout(p=dropout))
    layers.append(nn.Linear(1858592,10))

    self.layers = nn.Sequential(*layers)
        
    self.loss = nn.CrossEntropyLoss()

  def forward(self,x):
    return self.layers(x)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr= self.learning_rate)

  def training_step(self,batch):
    trainX,trainY = batch
    output = self(trainX)
    loss = self.loss(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()
    self.log('train_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('train_acc', acc,on_epoch = True,on_step = False,prog_bar=True)
    return loss

  # def on_train_epoch_end(self):
  #   train_acc = torch.stack([x['acc'] for x in self.output]).mean()
  #   print(train_acc)
  #   self.log('train_acc', train_acc, prog_bar=True)

  def validation_step(self, batch,batch_idx):
    trainX,trainY = batch
    output = self(trainX)
    loss = self.loss(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()
    self.log('val_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('val_acc', acc,on_epoch = True,on_step = False,prog_bar=True)
    return loss

  def test_step(self, batch,batch_idx):
    trainX,trainY = batch
    output = self(trainX)
    loss = self.loss(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()
    self.log('test_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('test_acc', acc,on_epoch = True,on_step = False,prog_bar=True)
    return loss


In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
import numpy as np
import random
transform = transforms.Compose([
          transforms.Resize((256,256)),
          # transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
      ])
train_dataset = datasets.ImageFolder(root=baseDir+'/train',
                                        transform=transform)

# index = np.arange(len(train_dataset))                       
# random.shuffle(index)
# train_dataset = torch.utils.data.Subset(train_dataset,index[:100])  
trainSize = int(0.8 * len(train_dataset))
valSize = len(train_dataset) - trainSize

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [trainSize, valSize])

train_dataset = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=32,
                                           shuffle=True,
                                           num_workers=1)

val_dataset = torch.utils.data.DataLoader(val_dataset,
                                           batch_size=32,
                                           shuffle=False,
                                           num_workers=1)

test_dataset = datasets.ImageFolder(root=baseDir+'/val',
                                        transform=transform)
test_dataset = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=32,
                                           shuffle=True,
                                           num_workers=1)


In [ ]:
filters = [64,64,64,64,32]
activation = nn.Mish()
activation_map = {"ReLU":nn.ReLU(), "GELU":nn.GELU(), "SiLU":nn.SiLU(), "Mish":nn.Mish()}
def sweeprun():
  wandb.init()
  filters = wandb.config.filters
  data_augmentation = wandb.config.data_augmentation
  batch_normalisation = wandb.config.batch_normalisation
  dropout = wandb.config.dropout
  activation = wandb.config.activation

  epochs = wandb.config.epochs
  learning_rate = wandb.config.learning_rate
  name = "lr_{}_ac_{}_dp_{}_bn_{}_fi_{}_da_{}".format(learning_rate,activation, dropout, batch_normalisation,filters, data_augmentation)
  activation = activation_map[activation]
  clf = CNN(filters,activation,batch_normalisation,dropout,learning_rate) 
  wandb_logger = WandbLogger(name = name)
  trainer = pl.Trainer(max_epochs=epochs,logger=wandb_logger) #,
  trainer.fit(clf,train_dataset,val_dataset)


In [ ]:
config= {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'val_accuracy'
      },
    "parameters":
    {

    "filters":{
      "values" :[[64,64,64,64,32]]
    },
    "data_augmentation" :{
        "values" : ["Yes","No"]
    },
    "batch_normalisation" :{
        "values" : ["Yes","No"]
    },
    "dropout" :{
        "values" : [0.2,0.3]
    },

    "activation" :{
          "values" : ["ReLU", "GELU", "SiLU", "Mish"]
    },
      "epochs" :{
          "values" : [5,  10]
    },

      "learning_rate" :{
          "values" : [1e-3,1e-4]
    }

    }

}

sweep_id = wandb.sweep(config,project="Assignment-02")

wandb.agent(sweep_id, sweeprun, count=3)

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


epoch,▁▁▅▅██
train_acc,▁▅█
train_loss,█▃▁
trainer/global_step,▁▁▅▅██
val_acc,█▁▃
val_loss,█▃▁
epoch,2
train_acc,0.30529
train_loss,44.11896
trainer/global_step,749
val_acc,0.1245


Error in callback <function _WandbInit._pause_backend at 0x7fa81c528f70> (for post_run_cell):


BrokenPipeError: ignored

# Pre - Trained

In [ ]:
import torch 
import pytorch_lightning as pl
import torch.nn as nn
from torch.nn  import functional
from torchvision import models

In [ ]:
class Resent(pl.LightningModule):
  def __init__(self):
    super(Resent,self).__init__()
    self.resnet = models.resnet18(pretrained=True)
    num_ftrs = self.resnet.fc.in_features
    self.resnet.fc = torch.nn.Linear(num_ftrs, 10) # replace 10 with number of output classes

  def forward(self,x):
      return self.resnet(x)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr = 0.001)

  def training_step(self,batch):
    trainX,trainY = batch
    output = self(trainX)
    loss = functional.cross_entropy(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()
    self.log('train_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('train_acc', acc,on_epoch = True,on_step = False,prog_bar=True)
    return loss



In [ ]:
# clf = Resent()

# trainer = pl.Trainer(max_epochs=10)
# trainer.fit(clf,data)

In [ ]:
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
import pytorch_lightning as pl

class MNISTNetwork(pl.LightningModule):
  
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(28 * 28, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 10),
      nn.Sigmoid()
    )
    self.bce = nn.BCELoss()
    
  def forward(self, x):
    return self.layers(x)
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    x = x.view(x.size(0), -1)
    y_hat = self.layers(x)
    loss = self.bce(y_hat, y)
    self.log('train_loss', loss)
    return loss
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
    return optimizer
  
if __name__ == '__main__':
  dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
  pl.seed_everything(42)
  neuralnetwork = MNISTNetwork()
  trainer = pl.Trainer()
  trainer.fit(neuralnetwork, DataLoader(dataset))


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 52.6 K
1 | bce    | BCELoss    | 0     
--------------------------------------
52.6 K    Trainable params
0         Non-trainable params
52.6 K    Total params
0.211     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

ValueError: ignored